In [9]:
import os
import redis

# Initialize Redis connection
r = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

try:
    from IPython.display import clear_output
except ImportError:
    clear_output = None

def clear_screen():
    if clear_output:
        clear_output(wait=True)  # Clear Jupyter notebook output
    else:
        os.system('clear')


In [10]:
# Funzione per mostrare gli utenti registrati
def mostra_utenti_registrati():
    utenti = r.smembers("utenti:registrati")
    if utenti:
        print("Utenti registrati disponibili:")
        for u in utenti:
            print(f" - {u}")
    else:
        print("Nessun utente registrato al momento.")



# Funzione per registrare nuovo utente
def registra_utente(username):
    email = input("Inserisci la tua email: ")
    password = input("Scegli una password: ")
    r.sadd("utenti:registrati", username)
    r.hset(f"utenti:{username}", mapping={
        "email": email,
        "password": password
    })
    print(f"Utente '{username}' registrato con successo.")
    return username

# Funzione di login
def login_utente(username):
    if not r.sismember("utenti:registrati", username):
        print("Utente non registrato.")
        return False
    password = input("Inserisci la password: ").strip()
    saved_password = r.hget(f"utenti:{username}", "password")
    if isinstance(saved_password, bytes):
        saved_password = saved_password.decode()

    if saved_password is None or saved_password != password:
        print("Password errata.")
        return False

    print(f"Login effettuato. Benvenuto {username}!")
    return True



def elimina_tutti_gli_utenti():
    r.flushdb()
    print("Tutti gli utenti sono stati eliminati.")


In [11]:
# ---- ToDo Functions ----
# Funzioni Todo
def add_todo(username):
    todo_id = r.incr(f"todo_id:{username}")  # ID incrementale per l'utente
    descrizione = input("Descrizione del Todo: ").strip()
    r.sadd(f"todo_utenti:{username}", todo_id)
    r.hset(f"todo_dettagli:{username}:{todo_id}", mapping={
        "description": descrizione,
        "completed": "0"
    })
    print("Todo aggiunto!")

def list_todos(username):
    todo_ids = r.smembers(f"todo_ids:{username}")
    if not todo_ids:
        print(f"Nessun todo per {username}.")
        return
    
    print(f"Todo per {username}:")
    for tid in todo_ids:
        # Se tid è bytes, decodificalo, altrimenti usalo direttamente
        if isinstance(tid, bytes):
            tid_str = tid.decode()
        else:
            tid_str = tid
        
        data = r.hgetall(f"todo_dettagli:{username}:{tid_str}")
        
        # Anche le chiavi/valori di data sono bytes, quindi decodifichiamoli
        desc = data.get(b"description", b"").decode() if b"description" in data else ""
        stato = data.get(b"stato", b"").decode() if b"stato" in data else ""
        
        print(f"- {tid_str}: {desc} (stato: {stato})")


def complete_todo(username):
    list_todos(username)
    tid = input("Inserisci l'ID del todo da completare: ").strip()
    if r.exists(f"todo_dettagli:{username}:{tid}"):
        r.hset(f"todo_dettagli:{username}:{tid}", "completed", "1")
        print("Todo segnato come completato.")
    else:
        print("Todo ID non valido.")

In [12]:
def menu():
    clear_screen()
    print("Benvenuto nel sistema di gestione utenti e ToDo!")
    print("Per favore, scegli un'opzione dal menu principale.")

    while True:
        print("Scegli un'opzione:")
        print("1. Mostra utenti registrati")
        print("2. Registra nuovo utente")
        print("3. Login utente")
        print("4. Elimina tutti gli utenti")
        print("5. Esci")    
        scelta = input("Inserisci il numero dell'opzione: ").strip()

        if scelta == "1":
            clear_screen()
            mostra_utenti_registrati()
            input("\nPremi invio per tornare al menu...")

        elif scelta == "2":
            username = input("Inserisci il tuo nome utente: ").strip()
            if r.sismember("utenti:registrati", username):
                print("Questo username è già registrato.")
                input("Premi invio per tornare al menu...")
                continue
            registra_utente(username)

            while True:
                sotto_scelta = input("\nVuoi effettuare subito il login? (1 = Sì, 2 = No): ").strip()
                if sotto_scelta == "1":
                    clear_screen()
                    if login_utente(username):
                        todo_menu(username)
                    break
                elif sotto_scelta == "2":
                    break
                else:
                    print("Scelta non valida. Riprova.")
                    input("Premi invio per riprovare...")
                    clear_screen()

        elif scelta == "3":
            username = input("Inserisci il tuo nome utente: ").strip()
            clear_screen()
            # Provo il login, se va bene entro nel todo_menu
            if login_utente(username):
                todo_menu(username)

        elif scelta == "4":
            conferma = input("Sei sicuro di voler eliminare tutti gli utenti? (s/n): ").strip().lower()
            if conferma == "s":
                elimina_tutti_gli_utenti()
                input("\nPremi invio per tornare al menu...")
            else:
                print("Operazione annullata.")
                input("Premi invio per tornare al menu...")

        elif scelta == "5":
            print("Uscita dal programma.")
            break
        
        else:
            print("Scelta non valida. Riprova.")
            input("Premi invio per riprovare...")

def todo_menu(username):
    while True:
        print(f"Menu Todo - Utente: {username}")
        print("1. Visualizza Todo")
        print("2. Aggiungi Todo")
        print("3. Completa Todo")
        print("4. Logout")

        scelta = input("Scegli un'opzione: ").strip()
        if scelta == "1":
            list_todos(username)
            input("\nPremi invio per tornare al menu Todo...")
        elif scelta == "2":
            add_todo(username)
            input("\nPremi invio per tornare al menu Todo...")
        elif scelta == "3":
            complete_todo(username)
            input("\nPremi invio per tornare al menu Todo...")
        elif scelta == "4":
            print("Logout effettuato.")
            break
        else:
            print("Opzione non valida.")
            input("Premi invio per riprovare...")



In [ ]:
# Avvia il programma
if __name__ == "__main__":
    menu()

Utenti registrati disponibili:
 - 3
Scegli un'opzione:
1. Mostra utenti registrati
2. Registra nuovo utente
3. Login utente
4. Elimina tutti gli utenti
5. Esci
Scelta non valida. Riprova.
Scegli un'opzione:
1. Mostra utenti registrati
2. Registra nuovo utente
3. Login utente
4. Elimina tutti gli utenti
5. Esci
Scelta non valida. Riprova.
Scegli un'opzione:
1. Mostra utenti registrati
2. Registra nuovo utente
3. Login utente
4. Elimina tutti gli utenti
5. Esci
Scelta non valida. Riprova.
Scegli un'opzione:
1. Mostra utenti registrati
2. Registra nuovo utente
3. Login utente
4. Elimina tutti gli utenti
5. Esci
Scelta non valida. Riprova.
Scegli un'opzione:
1. Mostra utenti registrati
2. Registra nuovo utente
3. Login utente
4. Elimina tutti gli utenti
5. Esci
Scelta non valida. Riprova.
Scegli un'opzione:
1. Mostra utenti registrati
2. Registra nuovo utente
3. Login utente
4. Elimina tutti gli utenti
5. Esci
Scelta non valida. Riprova.
Scegli un'opzione:
1. Mostra utenti registrati
2. Re

# Provare Dopo

In [ ]:
def crea_progetto(nome_progetto, utenti):
    for utente in utenti:
        r.sadd(f"progetto:{nome_progetto}", utente)
    print(f"Progetto '{nome_progetto}' creato con membri: {', '.join(utenti)}")

def add_progetto_todo(nome_progetto, description):
    todo_id = r.incr(f"todo_progetto:{nome_progetto}:id")
    r.sadd(f"todo_progetto:{nome_progetto}", todo_id)
    r.hset(f"todo_dettagli_progetto:{nome_progetto}:{todo_id}", mapping={
        "description": description,
        "completed": 0
    })


In [ ]:
# Pubblica evento
def notifica_completamento(username, todo_id):
    r.publish("notifiche", f"{username} ha completato il todo {todo_id}!")

# Listener (in un altro processo o thread)
def ascolta_notifiche():
    pubsub = r.pubsub()
    pubsub.subscribe("notifiche")
    print("In ascolto...")
    for msg in pubsub.listen():
        if msg['type'] == 'message':
            print(f"📢 {msg['data']}")
